## Imports & Auth

In [2]:
import os
import io
import re
import sys
import json
import math
import copy
import time
import shlex
import shutil
import random
import pathlib
import subprocess
import traceback
import concurrent.futures

import docker
import nbformat
import gspread

from pathlib import Path
from functools import partial
from datetime import datetime
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Callable, Dict, List, Sequence, Iterable, Union

import pandas as pd
from rclone_python import rclone
from nbclient import NotebookClient
from rclone_python.remote_types import RemoteTypes

from google.auth import default
from google.oauth2 import service_account
from googleapiclient.discovery import build, Resource
from googleapiclient.http import BatchHttpRequest, MediaIoBaseDownload, MediaIoBaseUpload
from googleapiclient.errors import HttpError

/Users/nabeel/PycharmProjects/e2e_sanity_checks/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
SERVICE_ACCOUNT_FILE = 'turing-delivery-g-ga-e36eb2300714.json'

# Combine scopes for both Drive and Sheets
SCOPES = [
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/spreadsheets",
]

def authenticate_with_service_account():
    """Authenticate using a service account and return credentials."""
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=SCOPES
    )
    return creds

# Get the shared credentials object
credentials = authenticate_with_service_account()

## Setup

In [4]:
from rich.console import Console
from rich.theme import Theme

# @title Logger Configs
custom_theme = Theme({
    "info": "cyan",
    "warning": "magenta",
    "error": "bold red"
})
console = Console(theme=custom_theme)

class Logger:
  @staticmethod
  def log(message):
    console.print(message, style="info")

  def error(message):
    console.print(message, style="error")

In [5]:
# @title GoogleService Class
class GoogleService:

  @classmethod
  def extract_file_id(cls, url):
      patterns = [
          r"/spreadsheets/d/([^/]+)",
          r"/file/d/([^/]+)",     # Matches /file/d/{file_id}
          r"[?&]id=([^&]+)",       # Matches ?id={file_id} or &id={file_id}
          r"/drive/([^/?#]+)",     # Matches /drive/{file_id} and stops at /, ?, or #
          r"/folders/([^/]+)"      # Matches /folders/{folder_id}
      ]

      for pattern in patterns:
          match_ = re.search(pattern, url)
          if match_:
              return match_.group(1).strip()  

In [ ]:
# @title GoogleDrive Functionality
class GoogleDrive(GoogleService):
    
    service = build("drive", "v3", credentials=credentials)

    @classmethod
    def get_file_names_in_batch(cls, file_ids):
        """
        Retrieves the names of multiple files from Google Drive in a single batch request.
        
        Args:
            drive_service: An authenticated Google Drive API service object.
            file_ids: A list of file IDs.
            
        Returns:
            A dictionary mapping file IDs to their names.
        """
        file_names = []
    
        def callback(request_id, response, exception):
            """
            Callback function to process the result of each individual request.
            """
            if exception:
                print(f"Error for file ID {request_id}: {exception}")
                
                file_names.append(
                    {
                        'colab_id': request_id,
                        'colab_name': None
                    }
                )
            else:
                file_names.append(
                    {
                        'colab_id': request_id,
                        'colab_name': response.get('name')
                    }
                )
    
        # Create a batch request with the callback
        batch = cls.service.new_batch_http_request(callback=callback)
    
        # Add a 'files().get()' request for each file ID
        for file_id in file_ids:
            batch.add(
                cls.service.files().get(
                    fileId=file_id,
                    fields='name',
                    supportsAllDrives=True
                ),
                request_id=file_id  # Use the file ID to track each request
            )
    
        # Execute the batch request
        batch.execute()
    
        return file_names

In [7]:
# @title GoogleSheets Functionality
class GoogleSheet(GoogleService):

  # service = build("sheets", "v4")
  service = build("sheets", "v4", credentials=credentials)

  @classmethod
  def get_sheet_data(cls, sheet_id: str, tab_name: str, **kwargs):
    """
    Gets data from existing Google Sheet and returns it as Pandas DataFrame.

    Args:
        sheet_id: The ID of the existing Google Sheet.
        tab_name: The desired name for the new tab.
        filter_col [Optional]: column name to filter the data.
        filter_val [Optional]: value to filter the data on.
    """
    vals = (
        cls.service.spreadsheets()
        .values()
        .get(spreadsheetId=sheet_id, range=tab_name)
        .execute()
        .get("values", [])
    )
    if len(vals) > 0:
      header = vals[0]
      data_values = vals[1:]
      max_columns = min(len(header), len(data_values[0]))
      data_values = [row[:max_columns] for row in data_values]
      header = header[:max_columns]
      df = pd.DataFrame(data_values, columns=header)
      df.columns = [column.strip() for column in df.columns]
      filter_cols = [col.strip() for col in kwargs.keys()]
      if filter_cols:
        if all(col in df.columns for col in filter_cols):
          query = " & ".join([
              f"{col}=='{kwargs[col]}'"
              if isinstance(kwargs[col], str)
              else f"{col}=={kwargs[col]}"
              for col in filter_cols])
          df = df.query(query)
        else:
          missing_cols = [col for col in filter_cols if col not in df.columns]
          raise Exception(f"Could not find column(s) in the sheet. {missing_cols}")
      return df
    sheet_name = cls.get_spreadsheet_name_by_id(sheet_id)
    raise Exception(f"No data found in the Tab: {tab_name}. Sheet ID: {sheet_name}")


  @classmethod
  def tab_exists(cls, spreadsheet_id, tab_name):

    spreadsheet_metadata = cls.service.spreadsheets().get(
        spreadsheetId=spreadsheet_id,
        fields='sheets.properties'
    ).execute()

    sheets = spreadsheet_metadata.get('sheets', [])
    for sheet in sheets:
        properties = sheet.get('properties')
        if properties and (properties.get('title') == tab_name):
            return True
    return False


  @classmethod
  def add_dataframe_to_sheet(cls, spreadsheet_id, df, tab_name, valueInputOption='RAW', drop_duplicates_on=['sample_id']):
    """
    Adds a new tab to an existing Google Sheet and populates it with data from a Pandas DataFrame.

    Args:
        spreadsheet_id: The ID of the existing Google Sheet.
        df: The Pandas DataFrame to export.
        tab_name: The desired name for the new tab.
    """
    try:
      if cls.tab_exists(spreadsheet_id, tab_name):
        Logger.log(f"Tab '{tab_name}' already exists in the spreadsheet.")
        existing_df = cls.get_sheet_data(spreadsheet_id, tab_name)
        # TODO: Add dataframe validation check
        Logger.log(f"Existing Dataframe")
        Logger.log(existing_df.info())

        combined_df = pd.concat([df, existing_df], ignore_index=True)
        df_to_upload = combined_df.drop_duplicates(subset=drop_duplicates_on, keep='first', ignore_index=True)
        Logger.log(f"Combined Dataframe")
        Logger.log(df_to_upload.info())

      else:
        Logger.log(f"Tab '{tab_name}' does not exist in the spreadsheet. Creating a new tab.")
        requests = [{
            'addSheet': {
                'properties': {
                    'title': tab_name
                }
            }
        }]
        batch_update_body = {
            'requests': requests
        }
        response = cls.service.spreadsheets().batchUpdate(
            spreadsheetId=spreadsheet_id,
            body=batch_update_body
        ).execute()
        # Get the ID of the newly created sheet (optional, but useful)
        new_sheet_id = response.get('replies')[0].get('addSheet').get('properties').get('sheetId')
        Logger.log(f"Successfully added new tab: '{tab_name}' with ID: {new_sheet_id}")
        df_to_upload = df

      values = [df_to_upload.columns.tolist()] + df_to_upload.values.tolist()
      Logger.log(f"Uploading {len(df_to_upload)} rows to tab '{tab_name}'.")
      range_name = f"'{tab_name}'!A1" # Ensure tab name is quoted if it has spaces or special characters
      body = {
          'values': values
      }
      result = cls.service.spreadsheets().values().update(
          spreadsheetId=spreadsheet_id,
          range=range_name,
          valueInputOption=valueInputOption,
          body=body
      ).execute()

      Logger.log(f"{result.get('updatedCells')} cells updated in tab '{tab_name}'.")

    except HttpError as err:
      Logger.error(f"An error occurred: {err}")
      if err.resp.status == 400: # Bad Request, often due to sheet name already existing
        Logger.error("Error details: The tab name might already exist or the request is malformed.")
      elif err.resp.status == 403: # Forbidden, often due to incorrect permissions
        Logger.error("Error details: Check your API permissions or if the service account/user has access to the sheet.")
      elif err.resp.status == 404: # Not Found, often due to incorrect spreadsheet ID
        Logger.error("Error details: The spreadsheet ID might be incorrect.")


  @classmethod
  def get_spreadsheet_name_by_id(cls, spreadsheet_id):
      """
      Retrieves the name (title) of a Google Spreadsheet given its ID.

      Args:
          spreadsheet_id: The ID of the Google Spreadsheet.

      Returns:
          The title of the spreadsheet, or None if an error occurs or spreadsheet is not found.
      """
      try:
          # Use spreadsheets().get() to retrieve metadata
          # We only request the 'properties.title' field for efficiency
          spreadsheet_metadata = cls.service.spreadsheets().get(
              spreadsheetId=spreadsheet_id,
              fields='properties.title'
          ).execute()

          # Extract the title from the properties
          title = spreadsheet_metadata.get('properties', {}).get('title')
          return title
      except HttpError as error:
          print(f'An error occurred: {error}')
          if error.resp.status == 404:
              print(f"Spreadsheet with ID '{spreadsheet_id}' not found.")
          return None



  @classmethod
  def add_dropdown_to_range(cls, spreadsheet_id: str, sheet_id: str,
                            dropdown_options: list,
                            range_start_row: int, range_end_row: int,
                            range_start_col: int, range_end_col: int):
    requests = [
        {
            'setDataValidation': {
                'range': {
                    'sheetId': sheet_id,
                    'startRowIndex': range_start_row,
                    'endRowIndex': range_end_row,
                    'startColumnIndex': range_start_col,
                    'endColumnIndex': range_end_col
                },
                'rule': {
                    'condition': {
                        'type': 'ONE_OF_LIST',
                        'values': [{'userEnteredValue': option} for option in dropdown_options]
                    },
                    'strict': True,  # Users can only enter values from the list
                    'showCustomUi': True, # Show dropdown arrow
                }
            }
        }
    ]

    # --- Execute the batch update request ---
    try:
        body = {
            'requests': requests
        }
        response = cls.service.spreadsheets().batchUpdate(
            spreadsheetId=spreadsheet_id,
            body=body
        ).execute()
        print(f"Dropdown added to Sheet ID {sheet_id}, Range row{range_start_row+1}:row{range_end_row}.")
        # You can inspect the response for more details if needed
        # print(response)
    except Exception as e:
        print(f"An error occurred: {e}")


  @classmethod
  def get_sheet_id_by_name(cls, spreadsheet_id: str, tab_name: str):
      """
      Retrieves the numerical Sheet ID (gid) for a given tab name within a spreadsheet.

      Args:
          spreadsheet_id (str): The ID of the Google Spreadsheet.
          tab_name (str): The exact name (title) of the tab/sheet to find.

      Returns:
          int: The numerical sheet ID (gid) if found.
          None: If an error occurs, the spreadsheet is not found, or the tab name is not found.
      """
      try:
          # Use spreadsheets().get() to retrieve metadata
          # We only request 'sheets.properties' to get sheet IDs and titles efficiently
          spreadsheet_metadata = cls.service.spreadsheets().get(
              spreadsheetId=spreadsheet_id,
              fields='sheets.properties'
          ).execute()

          sheets = spreadsheet_metadata.get('sheets', [])
          for sheet in sheets:
              properties = sheet.get('properties')
              # Check if properties exist and if the title matches the tab_name
              if properties and properties.get('title') == tab_name:
                  return properties.get('sheetId') # Return the sheetId (gid)

          # If the loop completes, the tab was not found
          print(f"Tab '{tab_name}' not found in spreadsheet with ID '{spreadsheet_id}'.")
          return None
      except HttpError as error:
          if error.resp.status == 404:
              print(f"Spreadsheet with ID '{spreadsheet_id}' not found. Error: {error}")
          else:
              print(f'An HTTP error occurred: {error}')
          return None
      except Exception as e:
          print(f"An unexpected error occurred while retrieving sheet ID for tab '{tab_name}': {e}")
          return None

## Download APIs Code

In [716]:
def download_apis(VERSION="0.1.0", download_datasets=False):
    import io
    import os
    import sys
    import zipfile
    import shutil
    import re
    # from google.colab import auth
    from googleapiclient.discovery import build
    from googleapiclient.http import MediaIoBaseDownload

    drive_service = GoogleDrive.service
    # Version to download
    # VERSION = "0.0.rev22final" # Version of the API
    
    # Define paths
    CONTENT_DIR = os.path.join('clean_workspace', VERSION)
    if os.path.exists(CONTENT_DIR):
        os.remove(CONTENT_DIR)
    os.makedirs(CONTENT_DIR, exist_ok=True)
    
    APIS_DIR = os.path.join(CONTENT_DIR, 'APIs')
    DBS_DIR = os.path.join(CONTENT_DIR, 'DBs')
    SCRIPTS_DIR = os.path.join(CONTENT_DIR, 'Scripts')
    FC_DIR = os.path.join(CONTENT_DIR, 'Schemas')
    ZIP_PATH = os.path.join(CONTENT_DIR, f'APIs_V{VERSION}.zip')
    
    # Google Drive Folder ID where versioned APIs zip files are stored
    APIS_FOLDER_ID = '1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4'
    
    # List of items to extract from the zip file
    ITEMS_TO_EXTRACT = ['APIs/', 'DBs/', 'Scripts/']
    
    # Clean up existing directories and files
    for path in [APIS_DIR, DBS_DIR, SCRIPTS_DIR, FC_DIR, ZIP_PATH]:
        if os.path.exists(path):
            if os.path.isdir(path):
                shutil.rmtree(path)
            else:
                os.remove(path)
    
    # Authenticate and create the drive service
    # auth.authenticate_user()
    # drive_service = build('drive', 'v3')
    # drive_service
    # Helper function to download a file from Google Drive
    def download_drive_file(service, file_id, output_path, file_name=None, show_progress=True):
        """Downloads a file from Google Drive"""
        destination = output_path
        request = service.files().get_media(fileId=file_id)
        with io.FileIO(destination, 'wb') as fh:
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
                if show_progress:
                    print(f"Download progress: {int(status.progress() * 100)}%")
    
    
    # 1. List files in the specified APIs folder
    print(f"Searching for APIs zip file with version {VERSION} in folder: {APIS_FOLDER_ID}...")
    apis_file_id = None
    
    try:
        query = f"'{APIS_FOLDER_ID}' in parents and trashed=false"
        results = drive_service.files().list(q=query, fields="files(id, name)").execute()
        files = results.get('files', [])
        for file in files:
            file_name = file.get('name', '')
            if file_name.lower() == f'apis_v{VERSION.lower()}.zip':
                apis_file_id = file.get('id')
                print(f"Found matching file: {file_name} (ID: {apis_file_id})")
                break
    
    except Exception as e:
        print(f"An error occurred while listing files in Google Drive: {e}")
    
    if not apis_file_id:
        print(f"Error: Could not find APIs zip file with version {VERSION} in the specified folder.")
        sys.exit("Required APIs zip file not found.")
    
    # 2. Download the found APIs zip file
    print(f"Downloading APIs zip file with ID: {apis_file_id}...")
    download_drive_file(drive_service, apis_file_id, ZIP_PATH, file_name=f'APIs_V{VERSION}.zip')
    
    # 3. Extract specific items from the zip file to /content
    print(f"Extracting specific items from {ZIP_PATH} to {CONTENT_DIR}...")
    try:
        with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
            zip_contents = zip_ref.namelist()
    
            for member in zip_contents:
                extracted = False
                for item_prefix in ITEMS_TO_EXTRACT:
                  if member == item_prefix or member.startswith(item_prefix):
                        zip_ref.extract(member, CONTENT_DIR)
                        extracted = True
                        break
    
    except zipfile.BadZipFile:
        print(f"Error: The downloaded file at {ZIP_PATH} is not a valid zip file.")
        sys.exit("Invalid zip file downloaded.")
    except Exception as e:
        print(f"An error occurred during extraction: {e}")
        sys.exit("Extraction failed.")
    
    
    # 4. Clean up
    if os.path.exists(ZIP_PATH):
        os.remove(ZIP_PATH)
    
    # 5. Add APIs to path
    if os.path.exists(APIS_DIR):
        sys.path.append(APIS_DIR)
    else:
        print(f"Error: APIS directory not found at {APIS_DIR} after extraction. Cannot add to path.")
    
    # 6. Quick verification
    # Check for the presence of the extracted items
    verification_paths = [APIS_DIR, DBS_DIR, SCRIPTS_DIR]
    all_present = True
    print("\nVerifying extracted items:")
    for path in verification_paths:
        if os.path.exists(path):
            print(f"✅ {path} is present.")
        else:
            print(f"❌ {path} is MISSING!")
            all_present = False
    
    if all_present:
        print(f"\n✅ Setup complete! Required items extracted to {CONTENT_DIR}.")
    else:
        print("\n❌ Setup failed! Not all required items were extracted.")

    # 7. Generate Schemas

    # Add Scripts to path
    if os.path.exists(CONTENT_DIR):
        sys.path.append(CONTENT_DIR)
    else:
        print(f"Error: CONTENT_DIR directory not found at {CONTENT_DIR} after extraction. Cannot add to path.")
    
    from Scripts.FCSpec import generate_package_schema
    
    print("\nGenerating FC Schemas")
    os.makedirs(FC_DIR, exist_ok=True)
    
    
    # Iterate through the packages in the /content/APIs directory
    for package_name in os.listdir(APIS_DIR):
        package_path = os.path.join(APIS_DIR, package_name)
    
        # Check if it's a directory (to avoid processing files)
        if os.path.isdir(package_path):
            # Call the function to generate schema for the current package
            generate_package_schema(package_path, output_folder_path=FC_DIR)
    print(f"✅ Successfully generated {len(os.listdir(FC_DIR))} FC Schemas to {FC_DIR}")

    if download_datasets:
        def download_drive_folder(service, folder_id, destination_path):
            """
            Recursively downloads all files in a Google Drive folder using the `download_drive_file`
            """
            os.makedirs(destination_path, exist_ok=True)
        
            query = f"'{folder_id}' in parents and trashed=false"
            page_token = None
        
            while True:
                results = service.files().list(
                    q=query,
                    spaces='drive',
                    fields='nextPageToken, files(id, name, mimeType)',
                    pageToken=page_token
                ).execute()
        
                for item in results.get('files', []):
                    file_id = item['id']
                    file_name = item['name']
                    mime_type = item['mimeType']
        
                    if mime_type == 'application/vnd.google-apps.folder':
                        # Recursively download subfolders
                        new_path = os.path.join(destination_path, file_name)
                        print(f"Creating subfolder and downloading: {new_path}")
                        download_drive_folder(service, file_id, new_path)
                    else:
                        # Construct full file path and pass it as output_path
                        full_path = os.path.join(destination_path, file_name)
                        print(f"Downloading file: {file_name} to {full_path}")
                        download_drive_file(service, file_id, full_path, file_name=file_name, show_progress=False)
        
                page_token = results.get('nextPageToken', None)
                if not page_token:
                    break
        
        # --- Configuration for Dataset Download ---
        # This FOLDER_ID should contain the 'Quotewk.csv' file.
        FOLDER_ID = "1tZqZB1vAxp4TTxbPm6O2YjfkZD4FM-ml"
        # DATASET_FOLDER = "./workspace/Datasets"
        DATASET_FOLDER = os.path.join(CONTENT_DIR, 'workspace/Datasets')
        
        print(f"Starting download of folder {FOLDER_ID} to {DATASET_FOLDER}...")
        download_drive_folder(drive_service, FOLDER_ID, DATASET_FOLDER)
        print("Dataset download complete.")

        # --- Configuration for WS Dataset Download ---
        # This FOLDER_ID should contain the 'WS Multihop Datasets' file.
        WS_DATA_ID = "1kmXZ1oarBPlE0OQL52eGoc1xPbupJ1n9"
        WS_DATA_ZIP_PATH = os.path.join(CONTENT_DIR, 'WS_DATA.zip')
        
        print(f"Downloading WS Dataset zip file with ID: {WS_DATA_ID}...")
        download_drive_file(drive_service, WS_DATA_ID, WS_DATA_ZIP_PATH, file_name=f'WS_DATA.zip')
        print("Dataset download complete.")
        
        # Extract the Datasets
        WS_DATA_ZIP_PATH = os.path.join(CONTENT_DIR, 'WS_DATA.zip')
        with zipfile.ZipFile(WS_DATA_ZIP_PATH, 'r') as zip_ref:
            zip_ref.extractall(CONTENT_DIR)
        print(f"Extracted to {CONTENT_DIR}")
        
        # Moving 'file_dataset_pb2.py' to root directory
        src_path = os.path.join(CONTENT_DIR, 'WS_DATA', 'file_dataset_pb2.py')
        dst_path = os.path.join(CONTENT_DIR, 'file_dataset_pb2.py')
        
        if os.path.exists(src_path):
            shutil.move(src_path, dst_path)
            print(f"Moved {src_path} to {dst_path}")
        else:
            print(f"Source file not found: {src_path}")
        
        # Clean up
        if os.path.exists(WS_DATA_ZIP_PATH):
            os.remove(WS_DATA_ZIP_PATH)        

In [717]:
download_apis(download_datasets=True)

Searching for APIs zip file with version 0.1.0 in folder: 1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4...
Found matching file: APIs_V0.1.0.zip (ID: 1hLV2slrHhH0RquKU-8oWRJRs_nHh5CT_)
Download progress: 100%
Extracting specific items from clean_workspace/0.1.0/APIs_V0.1.0.zip to clean_workspace/0.1.0...

Verifying extracted items:
✅ clean_workspace/0.1.0/APIs is present.
✅ clean_workspace/0.1.0/DBs is present.
✅ clean_workspace/0.1.0/Scripts is present.

✅ Setup complete! Required items extracted to clean_workspace/0.1.0.

Generating FC Schemas
✅ notes_and_lists Schema generation complete: clean_workspace/0.1.0/Schemas/notes_and_lists.json


Processing mutation notes_and_lists.mutations.m01...
✅ notes_and_lists.mutations.m01 Schema generation complete: clean_workspace/0.1.0/MutationSchemas/m01/notes_and_lists.json

✅ google_maps Schema generation complete: clean_workspace/0.1.0/Schemas/google_maps.json


Processing mutation google_maps.mutations.m01...
✅ google_maps.mutations.m01 Schema generation

## Fetch & Download Colabs / Notebooks

In [199]:
sheet_id = "1o_AmMV1t08vZc2vE-bHKnG-r3CrFKvDLDpM2wWo4EU4"
data_tab = "auto_qc_data"

colabs_df = GoogleSheet.get_sheet_data(sheet_id, data_tab)


# # Any filtering
# filter_col = 'status'
# filter_val = 'Needs Fixes'

# colabs_df = colabs_df[colabs_df[filter_col]==filter_val]

colabs_df['colab_id'] = colabs_df['colab_url'].apply(GoogleService.extract_file_id)
colabs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013 entries, 0 to 1012
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sample_id  1013 non-null   object
 1   colab_url  1013 non-null   object
 2   status     1013 non-null   object
 3   colab_id   1013 non-null   object
dtypes: object(4)
memory usage: 31.8+ KB


In [200]:
colab_names = []
name_request_batch_size = 99
for start in range(0, len(colabs_df['colab_id']), name_request_batch_size):
    colab_names += GoogleDrive.get_file_names_in_batch(colabs_df['colab_id'].tolist()[start:start+name_request_batch_size])
colab_name_df = pd.DataFrame(colab_names)
colab_name_df = colab_name_df[~colab_name_df['colab_name'].isna()]
colabs_df = pd.merge(colab_name_df, colabs_df, on='colab_id')
colabs_df = colabs_df.drop_duplicates(['colab_id'])
colabs_df.info()

Error for file ID 1BBbINzDvVSzpevYmssBPlZfYtO9LbN4n: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files/1BBbINzDvVSzpevYmssBPlZfYtO9LbN4n?fields=name&alt=json returned "File not found: 1BBbINzDvVSzpevYmssBPlZfYtO9LbN4n.". Details: "[{'message': 'File not found: 1BBbINzDvVSzpevYmssBPlZfYtO9LbN4n.', 'domain': 'global', 'reason': 'notFound', 'location': 'fileId', 'locationType': 'parameter'}]">
Error for file ID 13Fj8v64Gy0nrsnv9yDVkZqPX9e2WQDye: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files/13Fj8v64Gy0nrsnv9yDVkZqPX9e2WQDye?fields=name&alt=json returned "File not found: 13Fj8v64Gy0nrsnv9yDVkZqPX9e2WQDye.". Details: "[{'message': 'File not found: 13Fj8v64Gy0nrsnv9yDVkZqPX9e2WQDye.', 'domain': 'global', 'reason': 'notFound', 'location': 'fileId', 'locationType': 'parameter'}]">
Error for file ID 18a31wqZnCs_zMN13Ls2TKKvQhN2VVWXH: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files/18a31wqZnCs_zMN13Ls2TKKvQhN2VVWXH?fields

## Orchestrate Notebook AutoRun

In [205]:
total_samples = len(colabs_df)
max_container = 600
max_batch_size = math.ceil(total_samples / max_container)
print(f'Total Batches: {max_container}\nMax Samples Per Batch: {max_batch_size}')

Total Batches: 600
Max Samples Per Batch: 2


In [206]:
api_version = '0.1.0'
notebooks = [{'path': notebook, 'api_version': api_version} for notebook in colabs_df['colab_id'].tolist()]
notebooks_df = pd.DataFrame(notebooks)
for idx, api_version in enumerate(set(notebooks_df['api_version'])):
    count_notebooks = len(notebooks_df[notebooks_df['api_version']==api_version])
    batches = []
    for idx in range(count_notebooks):
        batches.append(idx//max_batch_size)
    batch_ids = [f"{api_version}_{batch}" for batch in batches]
    notebooks_df.loc[notebooks_df['api_version'] == api_version, 'batch_id'] = batch_ids

notebooks_df.to_csv('execution_configs.csv', index=False)

notebooks_df = pd.merge(notebooks_df, colabs_df, left_on='path', right_on='colab_id')
run_identifiers = list(set(notebooks_df['batch_id']))
notebooks_df['batch_id'].value_counts()

batch_id
0.1.0_0      2
0.1.0_340    2
0.1.0_338    2
0.1.0_337    2
0.1.0_336    2
            ..
0.1.0_160    2
0.1.0_159    2
0.1.0_158    2
0.1.0_157    2
0.1.0_494    1
Name: count, Length: 495, dtype: int64

In [217]:
def download_notebook(file_id):
    """Downloads a Colab notebook (.ipynb) from Google Drive."""
    drive_service = GoogleDrive.service
    request = drive_service.files().get_media(fileId=file_id, supportsAllDrives=True)
    file_stream = io.BytesIO()
    downloader = MediaIoBaseDownload(file_stream, request)
    done = False
    while not done:
        _, done = downloader.next_chunk()
    file_stream.seek(0)
    return file_stream.read()


dd = download_notebook('1nyw5UrEA6np_IKooIjNVWKk4yyYgl7cY')

In [223]:
json.loads(dd)['cells'][0]['source']

['**Sample ID**: 691\n',
 '\n',
 '**Query**:\n',
 '\n',
 'Can you help me sort the replies from my latest email campaign?\n',
 '\n',
 '**DB Type**: Base Case\n',
 '\n',
 '**Case Description**:\n',
 '\n',
 'The user, a lawyer, has recently sent out a cold email campaign with the subject "Divorce Lawyer". They have received several replies and want to categorize them and forward the lists to the appropriate teams on Slack. In their Gmail inbox, a "Hot Lead" is a reply where the person asks for a consultation or discusses their case containing the keywords "urgent", "speak today" or "very interested". Anything else is a "Cold Lead" containing the keyword "not interested". In their Slack workspace, there are two public channels named "closing-people" and "sales-people". No messages about these leads have been posted to these channels yet.\n',
 '\n',
 '```\n',
 '<multiturn info>\n',
 'Campaign Name: The campaign subject is "Divorce Lawyer". (Information Gathering)\n',
 'Categorization Rule:

In [230]:
import sanity_orchestrator_with_download as orchestrator
try:
    start_time = datetime.now()
    run_name = f'sanity_check_{start_time.strftime("%Y%m%d_%H%M%S")}'
    orchestrator.run_orchestration(run_name, run_identifiers[:5])
    print(f"Finished Docker Run. Time Taken: {(datetime.now()-start_time).seconds} Seconds")
except (FileNotFoundError, FileExistsError, ConnectionError) as e:
    print(f"\n❌ A critical error occurred: {e}")

--- Step 1: Validating Host Environment ---
✅ Docker client connected.

--- Step 2: Preparing Host Directories ---
✅ Created log directory for this run at: /Users/nabeel/PycharmProjects/e2e_sanity_checks/execution_logs/sanity_check_20250815_193957
✅ Created result directory for this run at: /Users/nabeel/PycharmProjects/e2e_sanity_checks/results/sanity_check_20250815_193957

--- Step 4: Launching Containers in Parallel ---
  -> Launching container 'sanity_check_20250815_193957-0' for batch 0...
  -> Launching container 'sanity_check_20250815_193957-1' for batch 1...
  -> Launching container 'sanity_check_20250815_193957-2' for batch 2...
  -> Launching container 'sanity_check_20250815_193957-3' for batch 3...
  -> Launching container 'sanity_check_20250815_193957-4' for batch 4...

--- Step 5: Waiting for All Containers to Finish ---
  -> ✅ SUCCESS | Container 'sanity_check_20250815_193957-0' finished with exit code 0.
  -> ✅ SUCCESS | Container 'sanity_check_20250815_193957-1' finishe

# Process Results

In [231]:
output_dir = f'results/{run_name}'
output_files = os.listdir(output_dir)
complete_data = []
for file in output_files:
    full_path = Path(output_dir) / file
    with open(full_path, 'r') as f:
        complete_data += json.load(f)['result']
# Use json_normalize to flatten the data
sanity_df = pd.json_normalize(complete_data)
sanity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   notebook                    10 non-null     object
 1   no_action_script_success    10 non-null     bool  
 2   no_action_response          10 non-null     object
 3   with_action_script_success  10 non-null     bool  
 4   with_action_response        10 non-null     object
 5   contains_golden_answer      10 non-null     bool  
 6   contains_final_assert       10 non-null     bool  
dtypes: bool(4), object(3)
memory usage: 408.0+ bytes


In [232]:
sanity_df.head()

,notebook,no_action_script_success,no_action_response,with_action_script_success,with_action_response,contains_golden_answer,contains_final_assert
0,1fNlXFhd4nLk4Rg1pIiuVo45BO-_FmzWj,True,Block: # Final Assertion\nError Type: Assertio...,True,,False,True
1,14nilo8LiDc5q8ovgWkHWavARtSKPN1eA,True,Block: # Final Assertion\nError Type: Assertio...,True,Block: # Action\nError Type: NameError\nError ...,False,True
2,1nyw5UrEA6np_IKooIjNVWKk4yyYgl7cY,True,Block: # Final Assertion\nError Type: Assertio...,True,,False,True
3,1h2C3rMUeEB6Hvhj7HaA8-gYZaV0hpqCD,True,Block: # Final Assertion\nError Type: NameErro...,True,Block: # Final Assertion\nError Type: NameErro...,False,True
4,1sTaFANN1qm0NBYvHQpW9vhkzQszmh0JR,True,,True,,True,True


In [233]:
columns_set = [
    'sample_id', 
    'colab_url', 
    'colab_name', 
    'no_action_script_success',
    'no_action_response',
    'with_action_script_success',
    'with_action_response',
    'contains_golden_answer',
    'contains_final_assert',
    ]
merged_df = pd.merge(colabs_df, sanity_df, left_on='colab_id', right_on='notebook')[columns_set]
merged_df = merged_df.fillna("")
for col in ['with_action_response', 'no_action_response']:
    merged_df[col] = merged_df[col].apply(lambda x: x[:49999])
merged_df.head()

,sample_id,colab_url,colab_name,no_action_script_success,no_action_response,with_action_script_success,with_action_response,contains_golden_answer,contains_final_assert
0,1142_base_GC,https://drive.google.com/file/d/1UbL5jd_UFUpkh...,Agent-1142_base_GC-Initial_with_DB_Setup.ipynb,True,Block: # Final Assertion\nError Type: Assertio...,True,Block: # Action\nError Type: NameError\nError ...,False,True
1,531_base_GC,https://drive.google.com/file/d/1sdiaZ7lc8ZfFz...,Agent-531_base_GC-Initial_with_DB_Setup.ipynb,True,Block: # Final Assertion\nError Type: Assertio...,True,,False,True
2,1134_edge_2,https://colab.research.google.com/drive/1sTaFA...,Agent-1134_edge_2-Merged.ipynb,True,,True,,True,True
3,1065_edge_1,https://colab.research.google.com/drive/1MFMaM...,Agent-1065_edge_1-Merged.ipynb,True,,True,,True,True
4,Terminal_2155_base_GC,https://colab.research.google.com/drive/1fNlXF...,Agent-2155_base_GC-Initial_with_DB_Setup.ipynb,True,Block: # Final Assertion\nError Type: Assertio...,True,,False,True


In [234]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   sample_id                   10 non-null     object
 1   colab_url                   10 non-null     object
 2   colab_name                  10 non-null     object
 3   no_action_script_success    10 non-null     bool  
 4   no_action_response          10 non-null     object
 5   with_action_script_success  10 non-null     bool  
 6   with_action_response        10 non-null     object
 7   contains_golden_answer      10 non-null     bool  
 8   contains_final_assert       10 non-null     bool  
dtypes: bool(4), object(5)
memory usage: 568.0+ bytes


In [235]:
FA_FAILED_ASSERTION = 'FA Failed - Assertion Error'
IA_FAILED_ASSERTION = 'IA Failed - Assertion Error'
NON_ASSERTION_ERROR = 'Non Assertion Error'
NO_ERROR_FOUND = 'No Error Found'
UNDEFINED_ERROR = 'Undefined Error Type'

NEEDS_FIXES = 'Needs Fixes'
GOOD_TO_GO = 'Good To Go'
NEEDS_MANUAL_REVIEW = 'Needs Manual Review'
CHECK_NOT_EXECUTED = 'Check Not Executed'

def add_error_type(error_message):
    if error_message == "":
        return NO_ERROR_FOUND
    block = error_message.split('\n')[0].split(':')[-1].strip()
    error_type = error_message.split('\n')[1].split(':')[-1].strip()
    initial_assertion_header = 'Initial Assertion'
    final_assertion_header = 'Final Assertion'
    # Non Assertion Error
    if error_type != 'AssertionError':
        return NON_ASSERTION_ERROR
    if error_type == 'AssertionError':
        if initial_assertion_header.lower() in block.lower():
            return IA_FAILED_ASSERTION
        if final_assertion_header.lower() in block.lower():
            return FA_FAILED_ASSERTION
    return UNDEFINED_ERROR

def get_auto_qc_status(status_w_action, status_wo_action, contains_final_assert, contains_golden_answer):
    if NON_ASSERTION_ERROR in [status_w_action, status_wo_action]:
        return NEEDS_FIXES

    if IA_FAILED_ASSERTION in [status_w_action, status_wo_action]:
        return NEEDS_FIXES

    if FA_FAILED_ASSERTION in [status_w_action]:
        return NEEDS_FIXES

    
    if status_w_action == NO_ERROR_FOUND:
        if status_wo_action == NO_ERROR_FOUND:
            if contains_final_assert:
                return NEEDS_FIXES
            if not contains_final_assert and not contains_golden_answer:
                return NEEDS_FIXES
            return GOOD_TO_GO
        
        if status_wo_action == FA_FAILED_ASSERTION:
            return GOOD_TO_GO
        
    return ""


In [236]:
merged_df['Execution Status w/o Action'] = merged_df['no_action_response'].apply(add_error_type)
merged_df['Execution Status w Action'] = merged_df['with_action_response'].apply(add_error_type)
merged_df['Auto QC Status'] = merged_df.apply(lambda row: get_auto_qc_status(row['Execution Status w Action'], 
                                                                             row['Execution Status w/o Action'],
                                                                             row['contains_final_assert'],
                                                                             row['contains_golden_answer']), axis=1)
merged_df['Auto QC Status'].value_counts()

Auto QC Status
Needs Fixes    5
Good To Go     5
Name: count, dtype: int64

In [237]:
merged_df.head()

,sample_id,colab_url,colab_name,no_action_script_success,no_action_response,with_action_script_success,with_action_response,contains_golden_answer,contains_final_assert,Execution Status w/o Action,Execution Status w Action,Auto QC Status
0,1142_base_GC,https://drive.google.com/file/d/1UbL5jd_UFUpkh...,Agent-1142_base_GC-Initial_with_DB_Setup.ipynb,True,Block: # Final Assertion\nError Type: Assertio...,True,Block: # Action\nError Type: NameError\nError ...,False,True,FA Failed - Assertion Error,Non Assertion Error,Needs Fixes
1,531_base_GC,https://drive.google.com/file/d/1sdiaZ7lc8ZfFz...,Agent-531_base_GC-Initial_with_DB_Setup.ipynb,True,Block: # Final Assertion\nError Type: Assertio...,True,,False,True,FA Failed - Assertion Error,No Error Found,Good To Go
2,1134_edge_2,https://colab.research.google.com/drive/1sTaFA...,Agent-1134_edge_2-Merged.ipynb,True,,True,,True,True,No Error Found,No Error Found,Needs Fixes
3,1065_edge_1,https://colab.research.google.com/drive/1MFMaM...,Agent-1065_edge_1-Merged.ipynb,True,,True,,True,True,No Error Found,No Error Found,Needs Fixes
4,Terminal_2155_base_GC,https://colab.research.google.com/drive/1fNlXF...,Agent-2155_base_GC-Initial_with_DB_Setup.ipynb,True,Block: # Final Assertion\nError Type: Assertio...,True,,False,True,FA Failed - Assertion Error,No Error Found,Good To Go


In [238]:
merged_df.insert(loc=1, column='colab_id', value=merged_df['colab_url'].apply(GoogleDrive.extract_file_id))

In [159]:
merged_df = merged_df.fillna("")

In [160]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   sample_id                    2557 non-null   object
 1   colab_id                     2557 non-null   object
 2   colab_url                    2557 non-null   object
 3   colab_name                   2557 non-null   object
 4   no_action_script_success     2557 non-null   object
 5   no_action_response           2557 non-null   object
 6   with_action_script_success   2557 non-null   object
 7   with_action_response         2557 non-null   object
 8   contains_golden_answer       2557 non-null   object
 9   contains_final_assert        2557 non-null   object
 10  Execution Status w/o Action  2557 non-null   object
 11  Execution Status w Action    2557 non-null   object
 12  Auto QC Status               2557 non-null   object
dtypes: object(13)
memory usage: 259.8

In [197]:
output_tab = 'auto_qc_response_test'
GoogleSheet.add_dataframe_to_sheet(sheet_id, merged_df, output_tab, drop_duplicates_on = ['colab_id'])


Tab 'auto_qc_response_test' does not exist in the spreadsheet. Creating a new tab.

Successfully added new tab: 'auto_qc_response_test' with ID: 1275880329

Uploading 2557 rows to tab 'auto_qc_response_test'.

33254 cells updated in tab 'auto_qc_response_test'.

# Download All Colabs

In [98]:
SERVICE_ACCOUNT_FILE = 'turing-delivery-g-ga-e36eb2300714.json'

# Combine scopes for both Drive and Sheets
SCOPES = [
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/spreadsheets",
]

def authenticate_with_service_account():
    """Authenticate using a service account and return credentials."""
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=SCOPES
    )
    return creds

# Get the shared credentials object
credentials = authenticate_with_service_account()

# Build the Drive service using the credentials
drive_service = build("drive", "v3", credentials=credentials)

# Authorize gspread (for Sheets) using the same credentials
gspread_client = gspread.authorize(credentials)

print("✅ Successfully authenticated for both Google Drive and Google Sheets!\nUse drive_service and gspread_client variables")

REMOTE_NAME = "gdrive"

if not rclone.is_installed():
    !curl https://rclone.org/install.sh | sudo bash
    print("rclone Downloaded and Installed!")
else:
    print("rclone already installed")

# --- Define the configuration parameters for the rclone remote ---
# The keys here correspond to rclone's configuration options.
config = {
    "type": "drive",
    "scope": "drive.readonly",  # Use 'drive' for read/write or 'drive.readonly'
    "service_account_file": str(SERVICE_ACCOUNT_FILE),
    # Add other options here if needed (see advanced section below)
}

# Force recreate the remote to ensure it uses the correct service account file
if rclone.check_remote_existing(REMOTE_NAME):
    try:
        # Delete the existing remote using subprocess
        import subprocess
        result = subprocess.run(['rclone', 'config', 'delete', REMOTE_NAME], 
                              capture_output=True, text=True, input='y\n')
        if result.returncode == 0:
            print(f"Deleted existing remote '{REMOTE_NAME}' to refresh configuration")
        else:
            print(f"Warning: Could not delete existing remote: {result.stderr}")
    except Exception as e:
        print(f"Warning: Could not delete existing remote: {e}")

try:
    # The create_remote function writes to the rclone.conf file.
    rclone.create_remote(
        remote_name=REMOTE_NAME,
        remote_type=RemoteTypes.drive, # This is equivalent to "drive"
        **config # Pass the config dictionary as keyword arguments
    )
    print(f"Success! Remote '{REMOTE_NAME}' created with service account: {SERVICE_ACCOUNT_FILE}")

except Exception as e:
    print(f"An error occurred during rclone setup: {e}")
    # If it still fails, let's try to use the existing remote
    print(f"Will proceed with existing remote configuration")

✅ Successfully authenticated for both Google Drive and Google Sheets!
Use drive_service and gspread_client variables
rclone already installed
Deleted existing remote 'gdrive' to refresh configuration


[16:28:01] WARNING  The drive client id and the client secret have not been set. Using defaults.      ]8;id=88862;file:///Users/nabeel/PycharmProjects/e2e_sanity_checks/.venv/lib/python3.9/site-packages/rclone_python/rclone.py\rclone.py]8;;\:]8;id=120505;file:///Users/nabeel/PycharmProjects/e2e_sanity_checks/.venv/lib/python3.9/site-packages/rclone_python/rclone.py#112\112]8;;\

Success! Remote 'gdrive' created with service account: turing-delivery-g-ga-e36eb2300714.json


In [99]:
# Function to reset current directory
def reset_directory(DIR_TO_RESET):
    if Path(DIR_TO_RESET).exists():
        shutil.rmtree(DIR_TO_RESET)

    Path(DIR_TO_RESET).mkdir(parents=True, exist_ok=True)
    print(f"Directory {DIR_TO_RESET} Cleaned up!")

In [100]:
def download_colabs(
    colab_df,
    *,
    dest_dir: str = "notebooks",
    remote: str  = "gdrive:",
    concurrency: int = 16,
    show_progress: bool = True,
    reset_dest_directory: bool = True
):
    """
    Fetch every Colab notebook listed in `colab_df` via rclone backend copyid.

    Parameters
    ----------
    colab_df     : DataFrame with at least ['colab_id', 'colab_name']
    dest_dir     : local folder to place the .ipynb files (created if missing)
    remote       : rclone remote name that points to Google Drive
    concurrency  : number of parallel rclone workers ( -> xargs -P N )
    show_progress: if True add '--progress' to rclone so you see bars

    Returns
    -------
    DataFrame identical to `colab_df` plus a 'local_path' column
    """
    if remote.strip()[-1] != ":":
        remote += ":"

    if reset_dest_directory:
        reset_directory(dest_dir)
    if colab_df.empty:
        print("Nothing to download – DataFrame is empty.")
        return colab_df

    # ‣ 1.  Preparation -------------------------------------------------------
    dest = pathlib.Path(dest_dir)
    dest.mkdir(parents=True, exist_ok=True)

    ids   = colab_df["colab_id"].tolist()
    names = colab_df["colab_name"].fillna("").tolist()

    # Pre-add the local_path column so the caller can use it even if the
    # download later fails for some items.
    colab_df = colab_df.copy()
    colab_df["local_path"] = [
        str(dest / (n if n.endswith(".ipynb") else f"{n}.ipynb")) if n else
        str(dest / f"{fid}.ipynb")
        for fid, n in zip(ids, names)
    ]

    # Quick “before” summary
    print("─── Download plan ────────────────────────────────────────────")
    print(f"Target directory        : {dest.resolve()}")
    print(f"Remote (Drive)          : {remote}")
    print(f"Concurrent rclone jobs  : {concurrency}")
    print(f"Colabs to download      : {len(ids):,}")
    print("──────────────────────────────────────────────────────────────")

    # ‣ 2.  Build the xargs / rclone command ---------------------------------
    cmd = [
        "xargs", "-P", str(concurrency), "-I{}",      # fan-out
        "rclone", "backend", "copyid",
        "--drive-shared-with-me",
        *(["--progress"] if show_progress else []),
        remote,          # ① remote
        "{}",            # ② placeholder → file-ID
        f"{dest}/",      # ③ destination (trailing / keeps original names)
    ]

    id_stream = "\n".join(ids).encode()

    # ‣ 3.  Execute and time it ----------------------------------------------
    t0 = time.perf_counter()
    subprocess.run(cmd, input=id_stream, check=True)
    elapsed = time.perf_counter() - t0

    # ‣ 4.  Post-run summary -------------------------------------------------
    # Count how many of the expected files now exist locally
    ok = sum(pathlib.Path(p).exists() for p in colab_df["local_path"])

    size_str = ""
    try:
        total_bytes = sum(p.stat().st_size for p in dest.glob("*.ipynb"))
        size_str = f"  (≈{total_bytes/1e6:,.1f} MB)"
    except Exception:
        pass                                          # ignore permission errors

    print("\n─── Download completed ───────────────────────────────────────")
    print(f"Downloaded successfully   : {ok:,} / {len(ids):,}{size_str}")
    print(f"Elapsed time              : {elapsed:,.1f} s")
    if ok < len(ids):
        print("⚠️  Some notebooks may have failed – check the rclone logs.")
    print("──────────────────────────────────────────────────────────────")

    return colab_df

In [101]:
NOTEBOOKS_DIR = "notebooks"

colabs_down_df = download_colabs(
    colabs_df,
    dest_dir=NOTEBOOKS_DIR,
    remote=REMOTE_NAME,
    concurrency=16,
    show_progress=True
)

Directory notebooks Cleaned up!
─── Download plan ────────────────────────────────────────────
Target directory        : /Users/nabeel/PycharmProjects/e2e_sanity_checks/notebooks
Remote (Drive)          : gdrive:
Concurrent rclone jobs  : 16
Colabs to download      : 2,547
──────────────────────────────────────────────────────────────
Transferred:   	          0 B / 0 B, -, 0 B/s, ETA -
Elapsed time:         0.0sTransferred:   	          0 B / 0 B, -, 0 B/s, ETA -
Elapsed time:         0.0sTransferred:   	          0 B / 0 B, -, 0 B/s, ETA -
Elapsed time:         0.0sTransferred:   	          0 B / 0 B, -, 0 B/s, ETA -
Elapsed time:         0.0sTransferred:   	          0 B / 0 B, -, 0 B/s, ETA -
Elapsed time:         0.0sTransferred:   	          0 B / 0 B, -, 0 B/s, ETA -
Elapsed time:         0.0sTransferred:   	          0 B / 0 B, -, 0 B/s, ETA -
Elapsed time:         0.0sTransferred:   	          0 B / 0 B, -, 0 B/s, ETA -
Elapsed time:         0.0sTransferred:   	          0 B /

# Parse Colab

In [102]:
colabs_down_df = colabs_down_df[['colab_id', 'local_path']]
colabs_down_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2547 entries, 0 to 2879
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   colab_id    2547 non-null   object
 1   local_path  2547 non-null   object
dtypes: object(2)
memory usage: 59.7+ KB


In [103]:
colabs_down_df.head()

,colab_id,local_path
0,1eObP5DnvGXLj9hRh9hb-jV0Jrwg8edXa,notebooks/Agent-283-Edge-2.ipynb
1,1GOqNko1I5SIqnDrilM8KbhZHsPL37f0t,notebooks/Agent-108-Base.ipynb
2,145bFTPyeqF77isaP9uTU1f5v_hQqLv9u,notebooks/Agent-127-Base.ipynb
3,1ntAczmtrIzGQBKfh06og8mKtU0n4EnUK,notebooks/Agent-185-Base.ipynb
4,1NGSwUOLK_2reNfsLt7yzhmuCOA7GPEBm,notebooks/Agent-312-Base.ipynb


In [189]:
def contains_golden_answer(notebook_json):
    pattern = r"^##?\s*Golden\s*Answer"
    regex = re.compile(pattern, re.IGNORECASE)
    
    markdown_cells = [cell for cell in notebook_json['cells'] if cell['cell_type'] == 'markdown']
    for cell in markdown_cells:
        if any([regex.match(line) for line in cell['source']]):
            return True
    return False

def contains_final_assert(notebook_json):
    fa_block_found = False
    for cell in notebook_json['cells']:
        pattern = r"^##?\s*Final\s*Assertion"
        regex_fa = re.compile(pattern, re.IGNORECASE)
        
        pattern = r"^\s*assert\s+.+"
        regex_assert = re.compile(pattern)

        if cell['cell_type'] == 'markdown' and not fa_block_found:
            if any([regex_fa.match(line) for line in cell['source']]):
                fa_block_found = True
            continue
        if fa_block_found and cell['cell_type'] == 'code':
            if any([regex_assert.match(line) for line in cell['source']]) or any(['raise AssertionError' in line for line in cell['source']]):
                return True
    return False



In [190]:
def parse_colab(row):
    path = f"notebooks/{row['colab_name']}"
    try:
        with open(path, "r", encoding="utf-8") as f:
            notebook_json = json.load(f)
        return pd.Series((contains_final_assert(notebook_json), contains_golden_answer(notebook_json)))
    except:
        return pd.Series(("N/R", "N/R"))

In [187]:
parse_colab({'colab_name': 'Agent-2474_base_GC-Terminal-to-GeminiCLI-Initial_with_DB_Setup.ipynb'})

found FA


0     True
1    False
dtype: bool

In [176]:
path = f"notebooks/Agent-2474_base_GC-Terminal-to-GeminiCLI-Initial_with_DB_Setup.ipynb"

with open(path, "r", encoding="utf-8") as f:
    notebook_json = json.load(f)

In [181]:
notebook_json['cells'][-1]['source'][-1]
pattern = r"^\s*assert\s+.+"
regex_assert = re.compile(pattern)
bool(regex_assert.match(notebook_json['cells'][-1]['source'][-1]))

True

In [170]:
colabs_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2547 entries, 0 to 2879
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   colab_id    2547 non-null   object
 1   colab_name  2547 non-null   object
 2   sample_id   2547 non-null   object
 3   colab_url   2547 non-null   object
 4   status      2545 non-null   object
dtypes: object(5)
memory usage: 119.4+ KB


In [191]:
merged_df[['contains_final_assert', 'contains_golden_answer']] = merged_df.apply(parse_colab, axis=1)


In [192]:
merged_df[['contains_final_assert', 'contains_golden_answer']].value_counts()

contains_final_assert  contains_golden_answer
True                   False                     1571
False                  True                       629
True                   True                       343
N/R                    N/R                         12
False                  False                        2
Name: count, dtype: int64

In [117]:
colabs_down_df.columns = ['colab_id', 'colab_name', 'contains_final_assert', 'contains_golden_answer']
colabs_down_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2547 entries, 0 to 2879
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   colab_id                2547 non-null   object
 1   colab_name              2547 non-null   object
 2   contains_final_assert   2547 non-null   object
 3   contains_golden_answer  2547 non-null   object
dtypes: object(4)
memory usage: 99.5+ KB


In [ ]:
# output_tab = 'ga_fa_info'
# sheet_id = "15XdJpUXvy7NC9Wb4NprQydcdQ5rQYdkij-NDo1saVzk"
# GoogleSheet.add_dataframe_to_sheet(sheet_id, colabs_down_df, output_tab, drop_duplicates_on = ['colab_id'])


Tab 'ga_fa_info' does not exist in the spreadsheet. Creating a new tab.

Successfully added new tab: 'ga_fa_info' with ID: 735620463

Uploading 2547 rows to tab 'ga_fa_info'.

10192 cells updated in tab 'ga_fa_info'.

In [124]:
sheet_id = "15XdJpUXvy7NC9Wb4NprQydcdQ5rQYdkij-NDo1saVzk"
data_tab = "auto_qc_response"

current_qc_df = GoogleSheet.get_sheet_data(sheet_id, data_tab)
current_qc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   sample_id                    2557 non-null   object
 1   file_id                      2557 non-null   object
 2   colab_url                    2557 non-null   object
 3   colab_name                   2557 non-null   object
 4   no_action_script_success     2557 non-null   object
 5   no_action_response           2557 non-null   object
 6   with_action_script_success   2557 non-null   object
 7   with_action_response         2557 non-null   object
 8   golden_answer_sample         2557 non-null   object
 9   Execution Status w/o Action  2557 non-null   object
 10  Execution Status w Action    2557 non-null   object
 11  Auto QC Status               2557 non-null   object
dtypes: object(12)
memory usage: 239.8+ KB


In [127]:
current_qc_df = current_qc_df[['sample_id', 'file_id', 'colab_url', 'colab_name',
       'no_action_script_success', 'no_action_response',
       'with_action_script_success', 'with_action_response',
       'Execution Status w/o Action',
       'Execution Status w Action', 'Auto QC Status']]


In [133]:
colabs_down_df.columns = ['file_id', 'colab_name', 'contains_final_assert', 'contains_golden_answer']

In [140]:
final_df = pd.merge(current_qc_df, colabs_down_df[['file_id', 'contains_final_assert', 'contains_golden_answer']], on='file_id', how='left')

In [141]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   sample_id                    2557 non-null   object
 1   file_id                      2557 non-null   object
 2   colab_url                    2557 non-null   object
 3   colab_name                   2557 non-null   object
 4   no_action_script_success     2557 non-null   object
 5   no_action_response           2557 non-null   object
 6   with_action_script_success   2557 non-null   object
 7   with_action_response         2557 non-null   object
 8   Execution Status w/o Action  2557 non-null   object
 9   Execution Status w Action    2557 non-null   object
 10  Auto QC Status               2557 non-null   object
 11  contains_final_assert        2545 non-null   object
 12  contains_golden_answer       2545 non-null   object
dtypes: object(13)
memory usage: 259.8